#### 작은 도움이라도 될 수 있길 바라며 내가 아는만큼 최대한 자세하게 적으려고 노력했어... 
#### 설명은 코드아래에 명시해 놓았으니 참고해!

In [1]:
import tensorflow as tf
import numpy as np
import tqdm

#### 관련된 라이브러리 import
  - tensorflow랑 numpy는 알 것 같아서 따로 안적음.
  - tqdm은 learning 시에 progress bar를 표현해줌.(원작자가 해놓아서 수정하지 않음)

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

#### dataset은 mnist를 활용하였음. mnist data set의 input_data를 import하는 구문

In [5]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


#### import된 input_data를 mnist 변수에 할당하고 주어진 경로 "./mnist/data/"에 저장함
  - one_hot = True 옵션은 label(0~9의 10개)를 0과 1로 치환해주는 코드임.(ex : 2 -> 0010000000, 7 -> 0000000100)
  - WARNING은 새로운 버전의 mnist data set이 있다는 안내이니 무시해도 무방

In [6]:
training_images, training_labels, test_images, test_labels = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
training_images = training_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

#### mnist data set을 training/test image와 label로 나누어 변수에 할당함.
#### training/test_images는 28 X 28 size의 picture 이나, 변수 할당 시 [-1, 786(28 X 28)]로 변환됨. 따라서 image로 활용하기 위해(ResNet이 이미지를 분할하는 concept 이므로) reshape함.( -1 : n개의 자료, 28, 28 : size, 1 : color(black/white)

In [14]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, 10])

#### input되는 자료를 할당하기 위한 placeholder 지정
  - x : image, y : label이 들어가므로 위의 수치가 이해될 것임, None은 n개라는 의미

In [8]:
def ResNET(input) :
    L = tf.nn.conv2d(input, tf.Variable(tf.random_normal([3, 3, 32, 32])), strides = [1, 1, 1, 1], padding = 'SAME')
    L = tf.nn.relu(L)
    L = tf.nn.conv2d(L, tf.Variable(tf.random_normal([3, 3, 32, 32])), strides = [1, 1, 1, 1], padding = 'SAME')
    L = input + L
    L = tf.nn.relu(L)
    return L

#### ResNet concept 구현을 위한 함수 선언
  - conv2d는 convolutional layer 구현을 위한 tool로 3 X 3 size인 32개의 filter를 활용, stride는 건너뛰는 값(1X1), padding이 same일 경우 stride가 1x1일 때, input과 output의 사이즈가 동일함(?, 28, 28, 32)
  - 이후, relu 통과
  - conv2d 재시행
  - 기존 input 값을 출력된 값이 더함(ResNET 핵심으로 gradient vanishing 방지, 깊은 layer를 쌓을 수 있으나, 여기서는 2단만 구현)

#### cnn 관련 이론은 다음의 동영상을 보면 이해될 것임. https://www.youtube.com/watch?v=Em63mknbtWo&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm&index=35

In [9]:
L = tf.nn.conv2d(x, tf.Variable(tf.random_normal([3, 3, 1, 32])), strides = [1,1,1,1], padding = 'SAME')
for i in range(5) :
    L = ResNET(L)

#### ResNet에 넣기위해 초기 Layer(L)을 3x3 fliter 32개를 활용하여 분할(하지만 padding 때문에 size는 같음)
#### ResNet에서 5번 반복됨.

In [15]:
last = tf.nn.conv2d(L, tf.Variable(tf.random_normal([3, 3, 32, 1])), strides = [1,1,1,1], padding = 'SAME')
print(last) # last shape chk'
flatten = tf.reshape(last, [-1, last.get_shape()[1:4].num_elements()])
W = tf.Variable(tf.random_normal([last.get_shape()[1:4].num_elements(), 10]))
result = tf.matmul(flatten, W)

Tensor("Conv2D_12:0", shape=(?, 28, 28, 1), dtype=float32)


#### last : fully conected layer에 연결하기 위해 마지막 flitering 후 출력layer를 1장으로 변경
#### flatten = last를 다시 [-1, n] 형태로 변환( n : 28 * 28 * 1)
#### W : softmax 활용을 위해 임의의 W값 지정 및 변수선언, output이 10종류 이므로 10
#### result : W * X로 이해하면 됨.(logit)

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = result, labels = y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(result, 1), tf.argmax(y, 1)), dtype = tf.float32))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

#### cost : softmax함수의 cost 함수인 cross entropy 사용
#### accuracy 계산은 예측값과 label 최대값의 index가 같은지(one_hot 이므로) 확인하고 true이면 1로 casting하여 평균을 구함.
  - 2번해서 1번 맞으면 (1 + 0) / 2로 구하는 식임.
#### optimizer는 Adam 사용(optimizer 고르는 건 솔직히 잘 모르겠음.)

In [17]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#### variable initializing, run session
#### 보기 불편해서 아래코드 설명은 여기다 적음
  - batch_size : 한번에 50개의 자료(그림)을 가져옴([j*batch_size:(j+1)*batch_size,:,:,:]중 뒤에 :, :, :은 잘 모르겠음ㅠㅠㅠ)
  - 첫번째 for roop는 epoch으로 10으로 임의지정(오래걸려서... 원작은 100)
  - 두번째 for roop는 실제 batch 및 training이 이루어짐.
  - cost는 epoch마다 list에 넣어서 평균으로 구함.
  - accuracy는 test dataset을 활용하여 구함.(training dataset 미활용)
  - accuracy가 떨어지는데... 원작자대로 하면 85%정도 나오는데... CPU로 돌리면 한시간 걸릴 것 같음. GPU 시도해봤는데 잘 안됨 ㅠㅠ
  - 참고한 블로그 주소 https://blog.naver.com/et2035/221371125386

In [13]:
batch_size = 50
cost_list = []
acc_list =[]
for i in range(10) :
    for j in tqdm.tqdm_notebook(range(5)) :
        _, c = sess.run([optimizer, cost], feed_dict = {x:training_images[j*batch_size:(j+1)*batch_size,:,:,:], y:training_labels[j*batch_size:(j+1)*batch_size,:]})
        cost_list.append(c)
    print(i+1, '-th Epoch\nCost :', np.mean(cost_list))
    print(i+1, '-th Epoch\nAccuracy : ', sess.run(accuracy, feed_dict={x:test_images, y:test_labels}))


1 -th Epoch
Cost : 4457044600000.0
1 -th Epoch
Accuracy :  0.1507



2 -th Epoch
Cost : 2760486700000.0
2 -th Epoch
Accuracy :  0.1786



3 -th Epoch
Cost : 2040985400000.0
3 -th Epoch
Accuracy :  0.1939



4 -th Epoch
Cost : 1615404700000.0
4 -th Epoch
Accuracy :  0.2021



5 -th Epoch
Cost : 1331382200000.0
5 -th Epoch
Accuracy :  0.2158



6 -th Epoch
Cost : 1127222000000.0
6 -th Epoch
Accuracy :  0.2387



7 -th Epoch
Cost : 974739000000.0
7 -th Epoch
Accuracy :  0.2536



8 -th Epoch
Cost : 856860600000.0
8 -th Epoch
Accuracy :  0.2533



9 -th Epoch
Cost : 763760740000.0
9 -th Epoch
Accuracy :  0.2605



10 -th Epoch
Cost : 688169750000.0
10 -th Epoch
Accuracy :  0.2698
